In [ ]:
from google.colab import files

upload = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download dataturks/face-detection-in-images

Dataset URL: https://www.kaggle.com/datasets/dataturks/face-detection-in-images
License(s): unknown
  0% 0.00/55.3k [00:00<?, ?B/s]
100% 55.3k/55.3k [00:00<00:00, 48.1MB/s]


In [ ]:
!ls

face-detection-in-images.zip  kaggle.json  sample_data


In [ ]:
!rm face-detection-in-images.zip

In [ ]:
!kaggle datasets download debanga/facial-expression-recognition-challenge

Dataset URL: https://www.kaggle.com/datasets/debanga/facial-expression-recognition-challenge
License(s): unknown
 95% 305M/321M [00:02<00:00, 122MB/s]
100% 321M/321M [00:02<00:00, 133MB/s]


In [ ]:
!ls

facial-expression-recognition-challenge.zip  kaggle.json  sample_data


In [ ]:
!unzip facial-expression-recognition-challenge.zip -d  facial-expression-recognition-challenge

Archive:  facial-expression-recognition-challenge.zip
  inflating: facial-expression-recognition-challenge/example_submission.csv  
  inflating: facial-expression-recognition-challenge/face_embs.npy  
  inflating: facial-expression-recognition-challenge/fer2013.tar  
  inflating: facial-expression-recognition-challenge/icml_face_data.csv/icml_face_data.csv  
  inflating: facial-expression-recognition-challenge/metadata_processed.csv  
  inflating: facial-expression-recognition-challenge/test.csv/test.csv  


In [ ]:
!ls facial-expression-recognition-challenge


example_submission.csv	fer2013.tar	    metadata_processed.csv
face_embs.npy		icml_face_data.csv  test.csv


In [ ]:
!wc -l facial-expression-recognition-challenge/metadata_processed.csv

35883 facial-expression-recognition-challenge/metadata_processed.csv


In [28]:
import pandas as pd

df = pd.read_csv('facial-expression-recognition-challenge/metadata_processed.csv')
emoticon = {'Angry': 0, 'Disgust' : 1, 'Fear' : 2, 'Happy' :3, 'Sad': 4, 'Surprise' : 5, 'Neutral': 6}
df['emotion'] = df['emotion'].map(emoticon)
df.head(5)

,label,image,emotion
0,4,59 71 89 118 139 160 183 222 212 148 77 16 16 ...,4
1,0,144 149 129 114 119 124 125 135 145 141 147 15...,0
2,4,90 97 105 42 41 41 37 40 36 55 104 128 145 162...,4
3,1,69 72 80 87 95 102 112 123 131 137 143 151 159...,1
4,0,134 135 136 137 142 122 46 47 36 25 21 18 13 1...,0


In [29]:
a = df.loc[0, 'image']
print(len(a.split(' ')))

2304


In [33]:
type(df.loc[0, 'image'])

str

str

In [34]:
import torch
import numpy as np

def process_image(image_string):
    # Split the string into a list of integers
    pixel_values = np.fromstring(image_string, dtype=int, sep=' ')

    # Reshape into a 48x48 array (assuming the images are 48x48)
    image_array = pixel_values.reshape(48, 48)

    # Convert to a PyTorch tensor
    tensor_image = torch.tensor(image_array, dtype=torch.float32)

    # Normalize the pixel values to the range [0, 1] if needed
    tensor_image /= 255.0

    return tensor_image

df['image'] = df['image'].apply(process_image)


In [35]:
df.head(5)

,label,image,emotion
0,4,"[[tensor(0.2314), tensor(0.2784), tensor(0.349...",4
1,0,"[[tensor(0.5647), tensor(0.5843), tensor(0.505...",0
2,4,"[[tensor(0.3529), tensor(0.3804), tensor(0.411...",4
3,1,"[[tensor(0.2706), tensor(0.2824), tensor(0.313...",1
4,0,"[[tensor(0.5255), tensor(0.5294), tensor(0.533...",0


In [37]:
df.drop('label', axis=1)

,image,emotion
0,"[[tensor(0.2314), tensor(0.2784), tensor(0.349...",4
1,"[[tensor(0.5647), tensor(0.5843), tensor(0.505...",0
2,"[[tensor(0.3529), tensor(0.3804), tensor(0.411...",4
3,"[[tensor(0.2706), tensor(0.2824), tensor(0.313...",1
4,"[[tensor(0.5255), tensor(0.5294), tensor(0.533...",0
...,...,...
35877,"[[tensor(0.0980), tensor(0.0824), tensor(0.082...",0
35878,"[[tensor(0.4588), tensor(0.4510), tensor(0.376...",0
35879,"[[tensor(0.7216), tensor(0.6980), tensor(0.600...",5
35880,"[[tensor(0.8039), tensor(0.8588), tensor(0.478...",2


In [38]:
def process_emotion(num):

    tensor_num = torch.tensor(num, dtype=torch.int32)

    return tensor_num

df['emotion'] = df['emotion'].apply(process_emotion)


In [39]:
df.head(2)

,label,image,emotion
0,4,"[[tensor(0.2314), tensor(0.2784), tensor(0.349...","tensor(4, dtype=torch.int32)"
1,0,"[[tensor(0.5647), tensor(0.5843), tensor(0.505...","tensor(0, dtype=torch.int32)"


In [48]:
from torch.utils.data import TensorDataset

image_tensors = torch.stack(df['image'].tolist())
label_tensors = torch.tensor(df['emotion'].tolist(), dtype=torch.long)

dataset = TensorDataset(image_tensors, label_tensors)

In [49]:
from torch.utils.data import DataLoader, random_split

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of testing samples: {len(test_dataset)}")

Number of training samples: 28705
Number of testing samples: 7177


In [50]:
import torch.nn as nn

model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(1*48*48, 128),
    nn.ReLU(),
    nn.Linear(128, 7)
)
model

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=2304, out_features=128, bias=True)
  (2): ReLU()
  (3): Linear(in_features=128, out_features=7, bias=True)
)

In [58]:
import torch

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
torch.manual_seed(1)
num_epochs = 100
log_epoch = 10

for epoch in range(num_epochs):
  accuracy_hist_train = 0
  for images, labels in train_dataloader:
    pred = model(images)
    loss = loss_fn(pred, labels)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    is_correct = (
        torch.argmax(pred, dim=1) == labels
        ).float()
    accuracy_hist_train += is_correct.sum()
  accuracy_hist_train /= len(train_dataloader.dataset)
  if (epoch % log_epoch == 0):
    print(f'Epoch {epoch + 1} Accuracy '
    f'{accuracy_hist_train:.4f}')

Epoch 1 Accuracy 0.8657
Epoch 11 Accuracy 0.8691
Epoch 21 Accuracy 0.8739
Epoch 31 Accuracy 0.8763
Epoch 41 Accuracy 0.8797
Epoch 51 Accuracy 0.8841
Epoch 61 Accuracy 0.8865
Epoch 71 Accuracy 0.8866
Epoch 81 Accuracy 0.8894
Epoch 91 Accuracy 0.8927


In [61]:
i = 0
for images, labels in test_dataloader:

  pred = model(images)
  is_correct = (
    torch.argmax(pred, dim=1) ==
    labels
    ).float()
  if i % 50 == 0:
    print(f'Test accuracy: {is_correct.mean():.4f}')
    print(labels)
  i += 1

Test accuracy: 0.3438
tensor([0, 5, 5, 2, 3, 5, 0, 4, 2, 6, 3, 6, 2, 1, 6, 5, 1, 1, 1, 2, 2, 3, 3, 5,
        5, 5, 0, 5, 2, 5, 2, 5])
Test accuracy: 0.3750
tensor([2, 5, 0, 2, 6, 4, 1, 5, 0, 4, 2, 6, 3, 4, 2, 3, 4, 5, 5, 1, 5, 5, 2, 4,
        4, 0, 5, 6, 2, 6, 3, 6])
Test accuracy: 0.2812
tensor([5, 2, 4, 6, 4, 3, 5, 3, 2, 4, 0, 4, 2, 5, 0, 6, 5, 3, 4, 5, 2, 4, 1, 0,
        1, 0, 1, 1, 1, 1, 5, 3])
Test accuracy: 0.2812
tensor([5, 5, 2, 6, 4, 3, 3, 5, 2, 0, 4, 2, 1, 4, 0, 4, 2, 6, 6, 6, 2, 0, 0, 2,
        5, 1, 2, 2, 3, 4, 1, 4])
Test accuracy: 0.3750
tensor([4, 3, 5, 1, 6, 3, 0, 0, 4, 4, 0, 5, 4, 1, 5, 2, 6, 5, 6, 5, 3, 4, 1, 2,
        3, 5, 6, 0, 2, 2, 2, 2])


In [62]:
upload = files.upload()

Saving sad_face.jpg to sad_face.jpg


In [69]:
from PIL import  Image
from torchvision import transforms

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

image = Image.open('sad_face.jpg')
image_tensor = transform(image).unsqueeze(0)  # Add batch dimension


output = model(image_tensor)
output

tensor([[  4.3259, -24.3329,   3.2990,  -3.0976,   2.5574,   8.6903,   9.4315]],
       grad_fn=<AddmmBackward0>)

In [70]:
model.eval()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=2304, out_features=128, bias=True)
  (2): ReLU()
  (3): Linear(in_features=128, out_features=7, bias=True)
)

In [71]:
torch.save(model.state_dict(), 'facial_emotion.ph')

In [72]:
!ls

facial_emotion.ph  facial-expression-recognition-challenge	kaggle.json   sample_data
facial_exp	   facial-expression-recognition-challenge.zip	sad_face.jpg


In [74]:
files.download('facial_emotion.ph')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>